In [1]:
%matplotlib inline
from __future__ import print_function, division
path = "/Users/trinakarmakar/anaconda2/data/fastai/state/"
sample_path = "/Users/trinakarmakar/anaconda2/data/fastai/state/sample/"
#path = "data/state/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using Theano backend.


In [2]:
batch_size = 64

In [3]:
batches = get_batches(sample_path+'train', batch_size=batch_size)
val_batches = get_batches(sample_path+'valid', batch_size=batch_size*2, shuffle=False)
test_batches = get_batches(sample_path+'test', batch_size=batch_size, shuffle=False)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 1 classes.


In [4]:
vgg = Vgg16()
model=vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [5]:
conv_model = Sequential(conv_layers)

In [6]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(sample_path)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 1 classes.


In [7]:
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

In [8]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_test_feat.dat', conv_test_feat)
save_array(path+'results/conv_feat.dat', conv_feat)

In [9]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_val_feat.shape

(1000, 512, 14, 14)

In [10]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [11]:
p=0.8

In [12]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

Train on 1500 samples, validate on 1000 samples
Epoch 1/1
1500/1500 [==============================] - 3s - loss: 4.7720 - acc: 0.0980 - val_loss: 5.1920 - val_acc: 0.1130 128/1500 [=>............................] - ETA: 2s - loss: 5.0622 - acc: 0.1094

In [14]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=2, 
             validation_data=(conv_val_feat, val_labels))

Train on 1500 samples, validate on 1000 samples
Epoch 1/2
1500/1500 [==============================] - 3s - loss: 4.2586 - acc: 0.0987 - val_loss: 3.6369 - val_acc: 0.0910 128/1500 [=>............................] - ETA: 2s - loss: 4.4092 - acc: 0.1172

In [15]:
bn_model.optimizer.lr=0.01

In [16]:
bn_model.save_weights(path+'models/conv8.h5')

In [17]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
da_batches = get_batches(sample_path+'train', gen_t, batch_size=batch_size, shuffle=False)

Found 1500 images belonging to 10 classes.


In [18]:
da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample*5)

In [19]:
save_array(path+'results/da_conv_feat2.dat', da_conv_feat)

In [20]:
da_conv_feat = np.concatenate([da_conv_feat, conv_feat])

In [21]:
da_trn_labels = np.concatenate([trn_labels]*6)

In [22]:
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [23]:
p=0.8

In [24]:
bn_model = Sequential(get_bn_da_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

Train on 9000 samples, validate on 1000 samples
Epoch 1/1
9000/9000 [==============================] - 28s - loss: 4.1488 - acc: 0.1194 - val_loss: 1.8355 - val_acc: 0.3680 128/9000 [..............................] - ETA: 26s - loss: 5.5050 - acc: 0.0703

In [26]:
bn_model.optimizer.lr=0.01

In [27]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

Train on 9000 samples, validate on 1000 samples
Epoch 1/4
9000/9000 [==============================] - 22s - loss: 3.0434 - acc: 0.1626 - val_loss: 1.6890 - val_acc: 0.5060 128/9000 [..............................] - ETA: 24s - loss: 3.4715 - acc: 0.1406

In [28]:
bn_model.optimizer.lr=0.0001

In [29]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

Train on 9000 samples, validate on 1000 samples
Epoch 1/4
9000/9000 [==============================] - 45s - loss: 1.7947 - acc: 0.3877 - val_loss: 1.3288 - val_acc: 0.8100

In [30]:
bn_model.save_weights(path+'models/da_conv8_1.h5')

Psedo Labeling:
   
We're going to try using a combination of pseudo labeling and knowledge distillation to allow us to use unlabeled data (i.e. do semi-supervised learning). For our initial experiment we'll use the validation set as the unlabeled data, so that we can see that it is working without using the test set. At a later date we'll try using the test set.
To do this, we simply calculate the predictions of our model...

In [31]:
val_pseudo = bn_model.predict(conv_val_feat, batch_size=batch_size)

...Concatenating with our training label

In [34]:
combo_pseudo = np.concatenate([da_trn_labels, val_pseudo])

In [35]:
comb_feat = np.concatenate([da_conv_feat, conv_val_feat])

In [ ]:
..and finetune our model using the data..

In [40]:
bn_model.load_weights(path+'models/da_conv8_1.h5')

In [44]:
bn_model.fit(comb_feat, combo_pseudo, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

Train on 10000 samples, validate on 1000 samples
Epoch 1/1
10000/10000 [==============================] - 61s - loss: 1.5687 - acc: 0.4921 - val_loss: 1.4821 - val_acc: 0.7800

In [46]:
bn_model.optimizer.lr=0.00001

In [48]:
bn_model.fit(comb_feat, combo_pseudo, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

Train on 10000 samples, validate on 1000 samples
Epoch 1/4
10000/10000 [==============================] - 53s - loss: 1.4850 - acc: 0.5274 - val_loss: 1.5142 - val_acc: 0.8230

In [49]:
bn_model.save_weights(path+'models/bn-ps8.h5')

Submit:
    
We need to find good clipping amount on validation set before submission.

In [ ]:
def do_clip(arr, mx):
    return np.clip(arr, (1-mx))/9, mx)

In [ ]:
keras.metrics.categorical_crossentropy(val_labels, do_clip(val_preds, 0.93)).eval()

In [ ]:
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

In [ ]:
preds = bn_model.predict(conv_test_feat, batch_size=batch_size*2)

In [ ]:
subm = do_clip(preds, 0.93)

In [ ]:
subm_name = path+'results\subm.gz'

In [ ]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)

In [ ]:
submission = pd.Dataframe(subm, columns=classes)
submission.insert(0, 'img', [a[4:] for a in test_filename])
submission.head()

In [ ]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [ ]:
FileLink(subm_name)